In [1]:
import pandas as pd
import json
import psycopg2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
def call_df(table_name):
    with open('config.json', 'r') as f:
        config = json.load(f)
        
    conn = psycopg2.connect(user = config['USER'],
                              password = config['PASSWORD'],
                              host = config['HOST'],
                              port = config['PORT'],
                              database = config['DATABASE'])
    
    sql = f'SELECT * FROM {table_name}'
    df = pd.read_sql_query(sql, conn)
    conn.close()
    return df

In [4]:
district = call_df('crawling_db.district_table')
apartment = call_df('crawling_db.apartment_table').drop(columns='table_id')
school = call_df('crawling_db.school_table').drop(columns='table_id')
subway = call_df('crawling_db.subway_table').drop(columns='table_id')
price = call_df('crawling_db.price_table').drop(columns='price_id')

In [5]:
df = (price.merge(apartment, how='left', on='apartment_id').
      merge(district, how='left', on='district_id').
      merge(school, how='left', on='apartment_id').
      merge(subway, how='left', on='apartment_id'))

In [6]:
import re
df['area'] = df['area'].apply(lambda x: int(re.split('\D',x)[0]))

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df['school_students'] = pd.to_numeric(df['school_students'])

In [ ]:
df.head()

# 연도별 거래액



In [ ]:
#2020년도의 경우 5월달까지의 거래액만 존재해서 제외
a = df[df['year']<2020]
figure,ax1 = plt.subplots()
figure.set_size_inches(18,6) 

sns.barplot(data=a,x="year",y="amount",ax=ax1)

- 2009년도부터 2013년까지의 실거래가가 감소하나, 2013년도부터 실거래가 증가

# 자치구별 거래액

- 실거래가를 자치구별로 나타내고, 이상치를 제거했다

In [ ]:
figure,ax1 = plt.subplots()
figure.set_size_inches(18,5)

sns.boxplot(data=df,y='amount',x='district_id')

- 이상치는 동일 면적의 해당 아파트 거래가 한번인 경우 제거하지 않았다
- 해당 아파트에 대한 거래 정보가 없어지기 때문

In [ ]:
#이상치 제거
df.drop(df[df['district_id']==2].sort_values(by='amount',ascending=False).head(1).index,inplace=True)
df.drop(df[df['district_id']==6].sort_values(by='amount',ascending=False).head(1).index,inplace=True)
df.drop(df[df['district_id']==12].sort_values(by='amount',ascending=False).head(5).index,inplace=True)
#df.drop(df[df['district_id']==13].sort_values(by='amount',ascending=False).head(2).index,inplace=True) 거래가 총 2번 이루어져서 제거 안함
df.drop(df[df['district_id']==14].sort_values(by='amount',ascending=False).head(1).index,inplace=True)
df.drop(df[df['district_id']==16].sort_values(by='amount',ascending=False).head(1).index,inplace=True)
df.drop(df[df['district_id']==18].sort_values(by='amount',ascending=False).head(1).index,inplace=True)
df.drop(df[df['district_id']==19].sort_values(by='amount',ascending=False).head(1).index,inplace=True)
df.drop(df[df['district_id']==22].sort_values(by='amount',ascending=False).head(3).index,inplace=True)#신호아파트 면적97인 아파트의 경우 한번 거래가 이루어져 제거 안함
df.drop(df[df['district_id']==24].sort_values(by='amount',ascending=False).head(1).index,inplace=True)

In [ ]:
# df[df['district_id']==22].sort_values(by='amount',ascending=False).head(4)[['apartment_name','area','amount']]

In [ ]:
# df[(df['apartment_name']=='신호')&(df['area']==132)]

In [ ]:
#이상치를 제거한 boxplot
figure,ax1 = plt.subplots()
figure.set_size_inches(18,5)

sns.boxplot(data=df,y='amount',x='district_id')

- 이상치를 제거한 이후의 자치구별 실거래가 boxplot

# 학교정보

In [ ]:
df['school_name'].nunique()

- 538개의 학교가 있으며, 초등학교 데이터만 존재 > 고등학교에 대한 데이터가 아니기 때문에 학교 이름에 따른 실거래가 영향은 없을 것으로 보임 > 제거

In [ ]:
a = df['school_addr_district']==df['district_id']
a = pd.DataFrame(a)
sns.heatmap(a,cbar=False, cmap="YlGnBu")

- 'school_add_district'와'district_id' 값이 모두 동일 > 제거한다 
- 행정동의 경우, 동일한 지역에 대해 다른 명칭으로 표기되어 있음(노량진동 노량진로) > 제거한다

In [ ]:
ind = df[df['school_name']=='서울가산초등학교'].index
df2 = df.drop(ind)
a = df[df['school_students']==0].index
df2 = df2.drop(a)
#서울가산초등학교의 경우 데이터에 따라 금천구 구로구에 모두 속한다고 
#표기되어있음. 우선 제거한다
#학생수가 0인 경우 또한 우선 제거해서 분석

In [ ]:
df2 = df2.groupby('school_name').mean()[['school_students','amount']].dropna()
df2 = pd.merge(df2,df[['district_name','school_name']],how='left',on='school_name').drop_duplicates()
df2.reset_index(inplace=True)
df2.head()
df2[df2['school_students']>2000] #학생수가 2000명 넘는 학교

- 학생수가 2000명 넘는 학교는 한군데, 서울대도초등학교 >> 제외하고 분석

In [ ]:
#대도초등학교 제외하고 분포 살펴보기
df2 = df2[df2['school_students']<2000]

- 자치구 세대수 대비 학교 학생수의 비율 구하기

In [ ]:
df2.head()

In [ ]:
population = pd.read_excel("population.xlsx")
population.drop(['※ 매년 말일자 통계 현황','Unnamed: 4'],axis=1,inplace=True)
population.drop([0,1,2],inplace=True)
population.rename(columns={'Unnamed: 1':'district_name','Unnamed: 2':'총인구수','Unnamed: 3':'세대수'},inplace=True)

In [ ]:
population['district_name'] = population.district_name.str.split(' ').str[1]
population['세대수'] = population.세대수.str.replace(',', '').astype('int64')

In [ ]:
population.head()

In [ ]:
df1 = pd.merge(df2,population,how='left',on='district_name')
df1['ratio'] = 0.0
for ind in df1.index:
  a = float(df1['세대수'][ind])
  b = float(df1['school_students'][ind])
  df1['ratio'][ind] = b/a
df1[df1['ratio']>0.015] #제거

- ratio는 각 자치구별 세대수와 학교 학생 수의 비율을 나타낸 것
- ratio는 0.00038~0.015사이
- ratio가 0.015 이상인 학교는 한 사례다 > 제거한다  

In [ ]:
df1 = df1[df1['ratio']<0.015]

In [ ]:
figure,ax1 = plt.subplots()
figure.set_size_inches(8,8) 

sns.regplot(x="ratio",y="amount",data=df1,color="m",ax=ax1)

- 자치구별 세대수대비 학교 학생수가 많을 수록 아파트의 실거래가가 높아진다

In [ ]:
factor = pd.cut(df1.ratio,5)
a = df1.amount.groupby(factor).mean()
a = pd.DataFrame(a)
a = a.reset_index()


figure,ax1 = plt.subplots()
figure.set_size_inches(18,5) 

sns.barplot(data=a,x="ratio",y="amount",ax=ax1)

- 자치구 세대수 인구 대비 학교 학생수의 비율이 커질수록, amount가 커지는 경향

# 세대당 주차대수

In [ ]:
figure,ax1 = plt.subplots()
figure.set_size_inches(18,6) 
sns.distplot(df['apartment_parking'],ax=ax1,bins=60)

- 세대당 주차대수는 0~2대 사이에 주로 분포

In [ ]:
df['apartment_parking_floor'] = df['apartment_parking'].apply(np.floor)

In [ ]:
figure,ax1 = plt.subplots()
figure.set_size_inches(18,5) 

sns.barplot(data=df,x="apartment_parking_floor",y="amount",ax=ax1)

- 세대당 주차대수에 따른 amount가 상관관계를 보이지 않는다. 들쑥날쑥..> 제거한다 

# 준공연도

In [ ]:
figure,ax1 = plt.subplots()
figure.set_size_inches(18,5) 

sns.pointplot(data=df,x="apartment_build_year",y="amount",ax=ax1)

In [ ]:
df['apartment_build_year'] = df['apartment_build_year'].apply(lambda x : x - x%10)
a = df['apartment_build_year'].value_counts()
a = pd.DataFrame(a)
a.reset_index(inplace=True)
a.rename(columns={'index':'apartment_build_year','apartment_build_year':'count'},inplace=True)


figure,(ax1,ax2) = plt.subplots(nrows=1,ncols=2)
figure.set_size_inches(18,6) 

sns.barplot(data=df,x="apartment_build_year",y="amount",ax=ax1)
sns.barplot(data=a,x="apartment_build_year",y="count",ax=ax2)

- 1970, 2020년대에 지어진 아파트의 수가 적은거에 비해 amount의 평균이 높다
- 2000년대에 가장 많이 아파트가 지어졌지만, 그에 비해 amount의 평균이 낮음
- 아파트 건축 연도를 일의 자리 수 제외하고 새롭게 설정

In [ ]:
df = df.drop(df[df.apartment_build_year==1930].index) #1930년에 지어진 아파트 제거

In [ ]:
figure,ax = plt.subplots()
figure.set_size_inches(18,6) 

sns.barplot(data=df,x="apartment_build_month",y="amount")

- apartment_build_month 제거

# 지하철역

In [ ]:
df['st_name'].nunique()

In [ ]:
figure,(ax1,ax2) = plt.subplots(nrows=2,ncols=1)
figure.set_size_inches(18,10) 

sns.barplot(data=df,y='amount',x='st_volume',ax=ax1)
sns.pointplot(data=df,y='amount',x='year',hue='st_volume',ax=ax2)

- 환승역이 4개인 곳의 실거래가가 2014년도 이후부터 꾸준히 증가
- 환승역의 개수 별 거래액이 큰 차이를 보이지 않는다 > 제거 

In [ ]:
#지하철역과의 거리
a = df.groupby('st_dist').mean()['amount']
a = pd.DataFrame(a).reset_index()

figure,ax1 = plt.subplots()
figure.set_size_inches(10,10) 

sns.regplot(x="st_dist",y="amount",data=a,color="m",ax=ax1)

- 가까운 역까지의 거리와 거래액은 음의 상관관계를 보인다 
- 이번에는 거리를 구간별로 나눠서 살펴보자

In [ ]:
factor = pd.cut(df.st_dist,7)
a = df.amount.groupby(factor).mean()
a = pd.DataFrame(a)
a = a.reset_index()

figure,ax1 = plt.subplots()
figure.set_size_inches(18,5) 

sns.barplot(data=a,x="st_dist",y="amount",ax=ax1)

- 거리가 0.0247 이하인 경우 지하철역에서 거리가 가까울수록 거래가가 비싸지만, 0.0247보다 큰 경우에는 양의 상관관계를 보인다
- 지하철역과 거리가 가장 먼 곳이 가장 높은 평균 거래가를 갖는다(평창동?을 예로 들 수 있을거 같다..자가용 이용..)

In [ ]:
df[df['st_dist']>0.0296].head()

- 실제로 아파트와 지하철과의 거리가 0.0296보다 먼 동네는 평창동이었다..

- 'apartment_build_month' 'school_name' 'school_addr_district' 'school_addr_town' 'st_name' 'apartment_floor_min', 'apartment_floor_max' 'apartment_parking' 'st_volume' >> 제거

In [ ]:
df = df.drop(['apartment_build_month','school_name','school_addr_district','school_addr_town','st_name','apartment_floor_min','apartment_floor_max','st_volume','period'],axis=1)
df.head()